# 特徴抽出器として用いる

## 入力ファイル準備

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
train_df = pd.read_csv("glue_data/ARD/train.tsv",delimiter="\t")

dev_df = pd.read_csv("glue_data/ARD/dev.tsv",delimiter="\t")

test_df = pd.read_csv("glue_data/ARD/test.tsv",delimiter="\t")

In [20]:
# trainだけで４万件あるので十分
train_df["reviewText"].to_csv("tmp/input.txt",index=None)

## 事前学習したモデルの出力を取得

コマンドラインで以下を実行
```zsh
export BERT_BASE_DIR=multilingual_L-12_H-768_A-12
python extract_features.py \
  --input_file=tmp/input.txt \
  --output_file=tmp/output.jsonl \
  --vocab_file=$BERT_BASE_DIR/vocab.txt \
  --bert_config_file=$BERT_BASE_DIR/bert_config.json \
  --init_checkpoint=$BERT_BASE_DIR/bert_model.ckpt \
  --layers=-1 \
  --max_seq_length=128 \
  --batch_size=8
  ```

## 出力されたファイルから取り出す

In [21]:
output = pd.read_json("tmp/output.jsonl",lines=True)

KeyboardInterrupt: 

## 学習器にかける

In [ ]:
X = output
y = train_df['helpful_rate']

In [ ]:
from sklearn.